<h1><strong>Análise de sentimentos em avaliações de e-commerce de roupas</strong></h1>


**Objetivo**:

Classificar as avaliações de e-commerce de roupas em positiva ou negativa.

*   Dataset: https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews

***Orientador:*** Thiago de Melo

***Orientando:*** Edson Brilhante

# Importando as bibliotecas

In [2]:
!pip install vaderSentiment

     |████████████████████████████████| 125 kB 8.1 MB/s 


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Importando o *dataset*

In [4]:
dataset = pd.read_csv('/content/drive/MyDrive/Edson Henrique/dataset/Womens Clothing E-Commerce Reviews.csv')

# Visualização do *dataset*

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


# Modelagem

## Seleciona as colunas '*Review Text*' e '*Rating*'

In [6]:
dataset = dataset[['Review Text', 'Rating']]
dataset = dataset.dropna()
dataset = dataset.reset_index(drop=True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22641 entries, 0 to 22640
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review Text  22641 non-null  object
 1   Rating       22641 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 353.9+ KB


## Encoding

Seleciona o atributo *'Rating'* e o converte da seguinte forma:



*   1, 2 e 3: negativa
*   4 e 5: positiva

In [7]:
dataset['Rating'] = dataset['Rating'].apply(lambda x: 'pos' if x >= 4 else 'neg')
dataset['Rating'] = pd.Categorical(dataset['Rating'])

In [8]:
dataset

,Review Text,Rating
0,Absolutely wonderful - silky and sexy and comf...,pos
1,Love this dress! it's sooo pretty. i happene...,pos
2,I had such high hopes for this dress and reall...,neg
3,"I love, love, love this jumpsuit. it's fun, fl...",pos
4,This shirt is very flattering to all due to th...,pos
...,...,...
22636,I was very happy to snag this dress at such a ...,pos
22637,"It reminds me of maternity clothes. soft, stre...",neg
22638,"This fit well, but the top was very see throug...",neg
22639,I bought this dress for a wedding i have this ...,neg


## Treino

In [11]:
dataset['vaderSentiment'] = 0
dataset['vaderSentiment Rating'] = np.nan
dataset

,Review Text,Rating,vaderSentiment,vaderSentiment Rating
0,Absolutely wonderful - silky and sexy and comf...,pos,0,NaN
1,Love this dress! it's sooo pretty. i happene...,pos,0,NaN
2,I had such high hopes for this dress and reall...,neg,0,NaN
3,"I love, love, love this jumpsuit. it's fun, fl...",pos,0,NaN
4,This shirt is very flattering to all due to th...,pos,0,NaN
...,...,...,...,...
22636,I was very happy to snag this dress at such a ...,pos,0,NaN
22637,"It reminds me of maternity clothes. soft, stre...",neg,0,NaN
22638,"This fit well, but the top was very see throug...",neg,0,NaN
22639,I bought this dress for a wedding i have this ...,neg,0,NaN


In [26]:
f_categorizar_vader = lambda x: 'pos' if x > 0.0 else 'neg'

analyzer = SentimentIntensityAnalyzer()

for i in range (0, dataset.shape[0]):
  vs = analyzer.polarity_scores(dataset['Review Text'].values[i])
  dataset.iloc[i,2] = vs['compound']
  dataset.iloc[i,3] = f_categorizar_vader(vs['compound'])
dataset

,Review Text,Rating,vaderSentiment,vaderSentiment Rating
0,Absolutely wonderful - silky and sexy and comf...,pos,0.8932,pos
1,Love this dress! it's sooo pretty. i happene...,pos,0.9729,pos
2,I had such high hopes for this dress and reall...,neg,0.9208,pos
3,"I love, love, love this jumpsuit. it's fun, fl...",pos,0.5727,pos
4,This shirt is very flattering to all due to th...,pos,0.9291,pos
...,...,...,...,...
22636,I was very happy to snag this dress at such a ...,pos,0.9152,pos
22637,"It reminds me of maternity clothes. soft, stre...",neg,0.3919,pos
22638,"This fit well, but the top was very see throug...",neg,0.9343,pos
22639,I bought this dress for a wedding i have this ...,neg,0.6692,pos


In [27]:
dataset['vaderSentiment Rating'] = pd.Categorical(dataset['vaderSentiment Rating'])

In [28]:
y_true = dataset['Rating'].values
y_pred = dataset['vaderSentiment Rating'].values

In [29]:
print('Matriz de confusão')
print(confusion_matrix(y_true, y_pred))

print('\nMétricas')
print(classification_report(y_true, y_pred, digits=3))

print('\nScore')
print(accuracy_score(y_true, y_pred))

Matriz de confusão
[[ 1106  4087]
 [  421 17027]]

Métricas
              precision    recall  f1-score   support

         neg      0.724     0.213     0.329      5193
         pos      0.806     0.976     0.883     17448

    accuracy                          0.801     22641
   macro avg      0.765     0.594     0.606     22641
weighted avg      0.788     0.801     0.756     22641


Score
0.8008921867408684


In [32]:
dataset[dataset['vaderSentiment Rating'] == 'pos']['vaderSentiment Rating'].count()

21114

In [33]:
dataset[dataset['vaderSentiment Rating'] == 'neg']['vaderSentiment Rating'].count()

1527